# Global molecular properties

This notebook focuses on atom- and bond-resolved properties parsed from Gaussian IRC output, including NBO charges and Wiberg bond indices.


## Setup

* Python 3.11
* Install project dependencies with `pip install -e .` from the repository root before running this notebook.

* Run the notebook from inside the `tutorials` directory (or the notebook's own folder) so Python picks up the installed `kudi` package from `src/`.


In [2]:

from kudi import IRCPath


from pathlib import Path


def find_fixture(filename: str = "output_sp.dat") -> Path:
    """Locate a tutorial fixture regardless of the working directory."""
    search_roots = [Path.cwd(), *Path.cwd().parents]
    for root in search_roots:
        for candidate in (root / "tutorials" / "fixtures" / filename, root / "fixtures" / filename):
            if candidate.exists():
                return candidate
    raise FileNotFoundError(
        f"Could not find {filename}; check that the tutorials/fixtures folder is available."
    )

fixture_path = find_fixture()
print(f"Using fixture: {fixture_path}")


Using fixture: /home/svogt/repos/kudi/tutorials/fixtures/output_sp.dat


## Parse the IRC path

All downstream examples reuse the same fixture file. The parser automatically collects NBO charge information and Wiberg bond orders when they are present in the log.


In [3]:

irc_path = IRCPath.from_file(fixture_path)
print(f"Loaded {len(irc_path.points)} points from {fixture_path.name}")


ParseError: Missing required IRC information in Gaussian block

## Natural bond orbital (NBO) charges

Use `IRCPath.nbo_charges` to collect charge series for specific atoms. The method returns a dictionary mapping atom labels to per-point values, with `None` where data are missing.


In [ ]:

charges = irc_path.nbo_charges(["O1", "C1"])

for label, series in charges.items():
    pretty = ", ".join("--" if v is None else f"{v:+.3f}" for v in series)
    print(f"{label:>3}: {pretty}")


## Wiberg bond indices

Bond orders are available through `IRCPath.wiberg_bond_orders`. Passing an explicit bond list limits the output to the bonds of interest.


In [ ]:

bond_orders = irc_path.wiberg_bond_orders(["C1-H2", "H3-O1"])

for bond, series in bond_orders.items():
    pretty = ", ".join("--" if v is None else f"{v:.3f}" for v in series)
    print(f"{bond:>6}: {pretty}")


## Bond-localized orbitals

When NBO bond orbitals are available, the `bond_orbitals` helper provides access to occupancy information. Each entry is a `NboBondOrbital` data class instance.


In [ ]:

bond_orbs = irc_path.bond_orbitals(["O1-H2"])
series = bond_orbs.get("O1-H2", [])
for idx, orb in enumerate(series):
    if orb is None:
        print(f"Point {idx}: no data")
    else:
        print(f"Point {idx}: {orb.kind} orbital {orb.key} with occupancy {orb.occupancy}")
